In [5]:
#import Files 
import xml.sax
import sys
import bz2

In [6]:
#global Variables 
PageNo=0 #to maintain total Number of pages in the dump

In [7]:
class parseXML( xml.sax.ContentHandler):
    
    global PageNo 
    def __init__(self):
        xml.sax.ContentHandler.__init__(self)
        self.current_tag=""
        self.title_content=""
        self.text_content=""
    
    def startElement(self,tag,attributes):
        self.current_tag=tag
        if tag=='page':
            self.title_content=""
            self.text_content=""

    def characters(self,content):
        if self.current_tag=="title":
            self.title_content+=content
        elif self.current_tag=="text":
            self.text_content+=content
            
    def endElement(self,tag):
        if tag=='page':
            global PageNo
            print("----------------------end page",PageNo,"----------------------------")
            PageNo+=1
            if PageNo==2:
                sys.exit()
        
        elif tag=='title':
            print("In title")
            print(self.title_content)
        
        elif tag=='text':
            print("In text")
        self.current_tag=""

In [8]:
if __name__=="__main__":
    Wiki_input_file="./Data/input.bz2"
    source = bz2.BZ2File(Wiki_input_file, "rb")

    #Create a parser Object
    parser= xml.sax.make_parser()

    #turnoff namespaces
    parser.setFeature(xml.sax.handler.feature_namespaces, 0)
    
    #callHandler
    handler= parseXML()
    
    #parse data 
    parser.setContentHandler(handler)
    parser.parse(source)
    
    

In title
AccessibleComputing
In text
----------------------end page 0 ----------------------------
In title
Anarchism
In text
----------------------end page 1 ----------------------------


SystemExit: 